# Project 1: Customer Database
**This is the first of three mandatory projects to be handed in as part of the assessment for the course 02807 Computational Tools for Data Science at Technical University of Denmark, autumn 2019.**

#### Practical info
- **The project is to be done in groups of at most 3 students**
- **Each group has to hand in _one_ Jupyter notebook (this notebook) with their solution**
- **The hand-in of the notebook is due 2019-10-13, 23:59 on DTU Inside**

#### Your solution
- **Your solution should be in Python**
- **For each question you may use as many cells for your solution as you like**
- **You should document your solution and explain the choices you've made (for example by using multiple cells and use Markdown to assist the reader of the notebook)**
- **You should not remove the problem statements, and you should not modify the structure of the notebook**
- **Your notebook should be runnable, i.e., clicking [>>] in Jupyter should generate the result that you want to be assessed**
- **You are not expected to use machine learning to solve any of the exercises**
- **You will be assessed according to correctness and readability of your code, choice of solution, choice of tools and libraries, and documentation of your solution**

## Introduction
Your team has been hired by the company X as data scientists. X makes gadgets for a wide range of industrial and commercial clients.

As in-house data scientists, your teams first task, as per request from your new boss, is to optimize business operations. You have decided that a good first step would be to analyze the companys historical sales data to gain a better understanding of where profit is coming from. It may also reveal some low hanging fruit in terms of business opportunities.

To get started, you have called the IT department to get access to the customer and sales transactions database. To your horror you've been told that such a database doens't exist, and the only record of sales transactions is kept by John from finance in an Excel spreadsheet. So you've emailed John asking for a CSV dump of the spreadsheet...

In this project you need to clean the data you got from John, enrich it with further data, prepare a database for the data, and do some data analysis. The project is comprised of five parts. They are intended to be solved in the order they appear, but it is highly recommended that you read through all of them and devise an overall strategy before you start implementing anything.

## Part 1: Cleaning the data
John has emailed you the following link to the CSV dump you requested.

- [transactions.csv](https://raw.githubusercontent.com/patrickcording/02807-comp-tools/master/docker/work/data/transactions.csv)

It seems as though he has been a bit sloppy when keeping the records. 

In this part you should:
- Explain what the data is
- Clean it to prepare it for inserting into a database and doing data analysis 

---
### Solution

In [72]:
import pandas as pd
import numpy as np
import re

In [143]:
df=pd.read_csv("https://raw.githubusercontent.com/patrickcording/02807-comp-tools/master/docker/work/data/transactions.csv",encoding="utf-8")

In [5]:
df.head()

,part,company,country,city,price,date
0,54868-5165,Chatterbridge,Spain,Barcelona,784.79€,2016-01-02 00:01:05
1,60505-2867,Lajo,Greece,Thessaloniki,187.99€,2016-01-02 00:05:26
2,24385-268,Flipstorm,Greece,Athens,221.73€,2016-01-02 00:18:30
3,76117-001,Twitterbeat,France,Annecy,1075.82€,2016-01-02 02:32:30
4,44946-1046,Chatterbridge,Spain,Barcelona,412.55€,2016-01-02 04:51:55


In [33]:
df.describe()

,part,company,country,city,price,date
count,20558,20568,18397,20535,20567,20568
unique,100,35,13,30,19214,20552
top,17156-617,Thoughtmix,Portugal,Amadora\t,-,10/04/2017
freq,300,2795,7383,2787,5,7


In [32]:
df.dtypes

part       object
company    object
country    object
city       object
price      object
date       object
dtype: object

In [36]:
for column in df.columns:
    print("Data type in {} is {}.".format(column,type(df[column][0])))

Data type in part is <class 'str'>.
Data type in company is <class 'str'>.
Data type in country is <class 'str'>.
Data type in city is <class 'str'>.
Data type in price is <class 'str'>.
Data type in date is <class 'str'>.


In [26]:
categorical=['company','country','city']
for column in categorical:
    print("" in df[column])

False
False
False


In [38]:
for column in df.columns:
    print("column {} has {} null value".format(column, df[column].isnull().sum().sum()))

column part has 10 null value
column company has 0 null value
column country has 2171 null value
column city has 33 null value
column price has 1 null value
column date has 0 null value


In [44]:
df.head()

,part,company,country,city,price,date
0,54868-5165,Chatterbridge,Spain,Barcelona,784.79€,2016-01-02 00:01:05
1,60505-2867,Lajo,Greece,Thessaloniki,187.99€,2016-01-02 00:05:26
2,24385-268,Flipstorm,Greece,Athens,221.73€,2016-01-02 00:18:30
3,76117-001,Twitterbeat,France,Annecy,1075.82€,2016-01-02 02:32:30
4,44946-1046,Chatterbridge,Spain,Barcelona,412.55€,2016-01-02 04:51:55


In [67]:
df['parsed_date'] = pd.to_datetime(df['date'],errors='coerce', infer_datetime_format=True)

In [68]:
df[df.parsed_date.isnull()]

,part,company,country,city,price,date,parsed_date
3539,17156-617,Thoughtmix,Portugal,Amadora\t,791.86€,2016-06-32 07:22:28,NaT
3540,54868-5165,Shufflebeat,Portugal,Porto,525.24€,2016-06-32 08:08:48,NaT


In [156]:
df2=df.dropna()

In [161]:
# I want to find all currency signs present
res=[]
for p in df2.price:
    for c in p:
        if c.isdigit() == False:
            res.append(c)
x=np.array(res)
np.unique(x)

array(['$', '-', '.', 'a', 'd', 'i', 'n', 'o', 'v', '£', '¥', '€'],
      dtype='<U1')

In [ ]:
        if c in ['a', 'd', 'i', 'n', 'o', 'v']:
            print(p)

In [149]:
re.findall("€","784.79€")

['€']

In [124]:
re.match("\d*\.?\d+", str('784.79€')).group(0)

'784.79'

In [120]:
def parse_price(string):
    amount=re.match('-?\d*\.?\d*', str(string))
    currency=re.match('[^-\d]',str(string))
    if currency:
        print(currency)
    if amount:
        return amount.group(0)
    else:
        return None

In [121]:
df2.price=df2.price.apply(parse_price)

In [119]:
df2.head()

,part,company,country,city,price,date,parsed_date
0,54868-5165,Chatterbridge,Spain,Barcelona,784.79,2016-01-02 00:01:05,2016-01-02 00:01:05
1,60505-2867,Lajo,Greece,Thessaloniki,187.99,2016-01-02 00:05:26,2016-01-02 00:05:26
2,24385-268,Flipstorm,Greece,Athens,221.73,2016-01-02 00:18:30,2016-01-02 00:18:30
3,76117-001,Twitterbeat,France,Annecy,1075.82,2016-01-02 02:32:30,2016-01-02 02:32:30
4,44946-1046,Chatterbridge,Spain,Barcelona,412.55,2016-01-02 04:51:55,2016-01-02 04:51:55


---
## Part 2: Enriching the data

A common task for a data scientists is to combine or enrich data from internal sources with data available from external sources. The purpose of this can be either to fix issues with the data or to make it easier to derive insights from the data.

In this part you should enrich your data with data from at least one external source. You may look to part 4 for some  inspiration as to what is required. Your solution should be automated, i.e., you can not ask the reader of your notebook to download any data manually. You should argue why and what you expect to achieve by the enrichments you are doing.

---
### Solution

---
## Part 3: Creating a database
Storing data in a relational database has the advantages that it is persistent, fast to query, and it will be easier access for other employees at Weyland-Yutani.

In this part you should:
- Create a database and table(s) for the data
- Insert data into the tables

You may use SQLite locally to do this. You should argue why you choose to store your data the way you do. 

---
### Solution

---
## Part 4: Analyzing the data
You are now ready to analyze the data. Your goal is to gain some actionable business insights to present to your boss. 

In this part, you should ask some questions and try to answer them based on the data. You should write SQL queries to retrieve the data. For each question, you should state why it is relevant and what you expect to find.

To get you started, you should prepare answers to the following questions. You should add more questions.
#### Who are the most profitable clients?
Knowing which clients that generate the most revenue for the company will assist your boss in distributing customer service ressources.

#### Are there any clients for which profit is declining?
Declining profit from a specific client may indicate that the client is disatisfied with the product. Gaining a new client is often much more work than retaining one. Early warnings about declining profit may help your boss fighting customer churn.


Remember, you are taking this to your new boss, so think about how you present the data.

---
### Solution

---
## Part 5: Performance
Your boss is very impressed with what you have achieved in less than two weeks, and he would like to take your idea of storing the customer and sales data in a relational database to production. However, John is concerned that the solution will not scale. His experience is telling him that you will see many occurrences of the following queries.

- Show all sales to company X between time $t_1$ and time $t_2$
- Show the latest X sales in the database
- Show total sales per company per day

Show that Johns concern is not justified.

---
### Solution

---